In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error

In [17]:
data_org = pd.read_csv("/content/drive/MyDrive/2022-1/2022-1 DScover/01. 태양광 프로젝트/태양광 프로젝트!/merged_final.csv")
data = data_org.copy()
data_1 = data.head(1)
data_1

,area,date,generation,avgTa,avgWs,avgRhm,ssDur,sumSsHr,sumGsr,avgTca,PM10,PM25
0,강원도,2016-01-01,144.042204,5.1,4.8,33.0,9.6,8.8,10.92,0.8,35.666667,27.298611


In [7]:
# data_1 = data_1.drop(['area','date', 'generation'], axis=1)

In [ ]:
# data_1_std_scaled = (data_1 - data_1.min(axis=0)) / (data_1.max(axis=0) - data_.min(axis=0))
# data_1_std_scaled

In [18]:
from sklearn.preprocessing import MinMaxScaler

# 원본 데이터 이상치 제거
data = data[(data['generation'] < 3000) & (data['generation'] >1)]
data = data[data['avgWs'] < 6]
data = data[data['PM10'] < 150]
data = data[data['PM25'] < 80]

data_used_to_fit_scale = data[['avgTa', 'avgWs','avgRhm','ssDur','sumSsHr','sumGsr','avgTca','PM10','PM25']]
data_used_to_fit_scale

scaler = MinMaxScaler()
print(scaler.fit(data_used_to_fit_scale))
print(scaler.data_max_)
#print(scaler.transform(data_used_to_fit_scale))

MinMaxScaler()
[ 33.9          5.96666667 100.          14.8         14.1
  39.72        10.         149.75833333  79.77777778]


In [13]:
# data_3의 형태로 데이터 프레임이 입력 된다고 가정 (예시임)
# 사용자가 지역을 선택하면서 자동으로 입력되는 지역 평균 데이터 + 사용자가 직접 입력한 데이터(지역, 연도, 월, 날짜, 기온, 미세먼지, 습도)
data_1['day'] = data_1['date'].str.split('-').str[2]
data_1['month'] = data_1['date'].str.split('-').str[1]
data_1['year'] = data_1['date'].str.split('-').str[0]
data_1 = data_1.drop(['date', 'generation'], axis=1)
data_1

,area,avgTa,avgWs,avgRhm,ssDur,sumSsHr,sumGsr,avgTca,PM10,PM25,day,month,year
0,강원도,5.1,4.8,33.0,9.6,8.8,10.92,0.8,35.666667,27.298611,01,01,2016


In [14]:
def encoding(df):
  # ohe - 날짜, 지역
  one_hot_encoded =pd.get_dummies(df, columns=['year', 'month', 'day', 'area'])
  one_hot_encoded = one_hot_encoded.drop(['avgTa', 'avgWs','avgRhm','ssDur','sumSsHr','sumGsr','avgTca','PM10','PM25'], axis=1)

  # min-max scale - 나머지 변수
  min_max_scale = df[['avgTa', 'avgWs','avgRhm','ssDur','sumSsHr','sumGsr','avgTca','PM10','PM25']]
  min_max_scaled = scaler.transform(min_max_scale)
  min_max_scaled = pd.DataFrame(data=min_max_scaled, columns=['avgTa', 'avgWs','avgRhm','ssDur','sumSsHr','sumGsr','avgTca','PM10','PM25'])

  # ohe + min-max scaled 하나의 데이터 프레임으로 합치기
  result = pd.concat( [one_hot_encoded, min_max_scaled], axis=1 )
  return result

In [15]:
encoding(data_1)
# 여기에 원본 데이터 ohe 시켰을 때 발생하는 행들 추가해야 할듯?

,year_2016,month_01,day_01,area_강원도,avgTa,avgWs,avgRhm,ssDur,sumSsHr,sumGsr,avgTca,PM10,PM25
0,1,1,1,1,0.408624,0.801136,0.252232,0.018868,0.624113,0.274559,0.08,0.229371,0.338382


In [ ]:
def rescale(df):
  # rescale 위해 generation 최대 최솟값 저장
  y_min = data_org['generation'].min()
  y_max = data_org['generation'].max()
  df = df[['avgTa', 'avgWs','avgRhm','ssDur','sumSsHr','sumGsr','avgTca','PM10','PM25']]
  rescaled = scaler.inverse_transform(df)
  rescaled = pd.DataFrame(data=rescaled, columns=['avgTa', 'avgWs','avgRhm','ssDur','sumSsHr','sumGsr','avgTca','PM10','PM25'])
  return rescaled

In [ ]:
rescale(data_1)

,avgTa,avgWs,avgRhm,ssDur,sumSsHr,sumGsr,avgTca,PM10,PM25
0,233.57,55.30,2967.20,60.38,124.08,433.544,8.0,8844.994927,3897.903791
1,428.37,56.45,3934.88,60.38,122.67,399.402,8.0,9402.865810,3766.039062
2,423.50,30.00,5010.08,60.38,101.52,344.616,8.0,14616.548015,6332.939670
